In [1]:
import pandas as pd
import json
import requests
import time
import re
from bs4 import BeautifulSoup

In [16]:
craigslist = pd.read_csv("craigslist_with_alias.csv")
craigslist.head(10)

,CarMake,CarModel,CarYear,CarPrice,CarCondition,MilesOnCar,PostingDate,PostingUrl,CarImageLinks,PostingRegion,Model,alias,Year
0,Chevrolet,Suburban High Country,2021,-1,NaN,4248,5/31/2021,https://spokane.craigslist.org/ctd/d/otis-orch...,['https://images.craigslist.org/00Q0Q_aECyW6g1...,Spokane / Coeur d'Alene,Suburban High Country,Suburban,2021
1,Nissan,Versa,2021,16950,excellent,4373,5/31/2021,https://seattle.craigslist.org/tac/ctd/d/tacom...,['https://images.craigslist.org/00X0X_laewhzXy...,Seattle-Tacoma,Versa,Versa,2021
2,Chevrolet,Equinox Lt,2021,33639,excellent,4,5/29/2021,https://kpr.craigslist.org/ctd/d/college-place...,['https://images.craigslist.org/00w0w_5zmhxECF...,Kennewick-Pasco-Richland,Equinox Lt,Equinox,2021
3,Chevrolet,Lifted Silverado 1500,2021,74999,NaN,467,5/31/2021,https://spokane.craigslist.org/ctd/d/spokane-l...,['https://images.craigslist.org/00g0g_gktrvbbL...,Spokane / Coeur d'Alene,Lifted Silverado 1500,Silverado 1500,2021
4,Nissan,Sentra Sr,2021,25505,excellent,5,5/29/2021,https://kpr.craigslist.org/ctd/d/college-place...,['https://images.craigslist.org/01616_kjQqonrj...,Kennewick-Pasco-Richland,Sentra Sr,Sentra,2021
5,Chevrolet,Silverado 1500 Lt,2021,55900,excellent,4,5/28/2021,https://kpr.craigslist.org/ctd/d/college-place...,['https://images.craigslist.org/00606_jqXcCiCI...,Kennewick-Pasco-Richland,Silverado 1500 Lt,Silverado 1500,2021
6,Chevrolet,Silverado 3500Hd Lt,2021,75465,excellent,4,5/28/2021,https://kpr.craigslist.org/ctd/d/college-place...,['https://images.craigslist.org/00B0B_5UUODsqG...,Kennewick-Pasco-Richland,Silverado 3500Hd Lt,Silverado 3500HD,2021
7,Toyota,Corolla,2021,22500,excellent,2522,5/31/2021,https://seattle.craigslist.org/tac/ctd/d/tacom...,['https://images.craigslist.org/01616_fGGvhfm4...,Seattle-Tacoma,Corolla,Corolla,2021
8,Chevrolet,Silverado 1500 High,2021,74999,NaN,467,5/24/2021,https://pullman.craigslist.org/ctd/d/spokane-l...,['https://images.craigslist.org/00g0g_gktrvbbL...,Pullman / Moscow,Silverado 1500 High,Silverado 1500,2021
9,Honda,Odyssey Exl,2021,69070,excellent,81,5/31/2021,https://skagit.craigslist.org/ctd/d/everett-20...,['https://images.craigslist.org/00707_ibgghsmT...,Skagit Island / San Juan Islands,Odyssey Exl,Odyssey,2021


In [3]:
craigslist_mmy = craigslist.loc[:, ["CarMake", "alias", "Year"]].copy()
craigslist_mmy = craigslist_mmy.drop_duplicates()

In [4]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

In [5]:
def clean_price(text):
        text = text.replace(",", "")
        text = re.findall("\\$\\d+", text)
        return text[0][1:]
            

In [6]:
def scrape_car_info(model, year):
    url1 = "https://www.carqueryapi.com/api/0.3/?callback=?&cmd=getTrims&model={}&year={}"
    url2 = "https://www.carqueryapi.com/api/0.3/?callback=?&cmd=getModel&model={}"
    
    data_kbb = {
            'Pros': "N/A",
            'Cons': "N/A",
            'StartingPrice': 0,
            'Review': "N/A",
            'New': "N/A"
    }
    data_car_query = {
        'OriginCountry': "make_country",
        'usSale': "model_sold_in_us",
        'bodyStyle': "model_body",
        'exteriorColors': "ExtColors",
        'interiorColors': "IntColors",
        'engineLocation': "model_engine_position",
        'engineType': "model_engine_type",
        'engineCylinders': "model_engine_cyl",
        'engineDisplacement_cc': "model_engine_cc",
        'engineDisplacement_I': "model_engine_l",
        'engineDisplacement_cubicIn': "model_engine_ci",
        'engineBore_mm': "model_engine_bore_mm",
        'engineBore_in': "model_engine_bore_in",
        'engineStroke_mm': "model_engine_stroke_mm",
        'engineStroke_in': "model_engine_stroke_mm",
        'valvePerCylinder': "model_engine_valves_per_cyl",
        'maxPower_hp': "model_engine_power_hp",
        'maxPower_ps': "model_engine_power_ps",
        'maxPower_kW': "model_engine_power_kw",
        'maxPower_rpm': "model_engine_power_rpm",
        'maxTorque_Nm': "model_engine_torque_nm",
        'maxTorque_Lb_ft': "model_engine_torque_lbft",
        'maxTorque_kgf_m': "model_engine_torque_kgm",
        'maxTorque_rpm': "model_engine_torque_rpm",
        'engineCompressionRatio': "model_engine_compression",
        'engineFuelType': "model_engine_fuel",
        'drive': "model_drive",
        'transmission': "model_transmission_type",
        'topSpeed_mph': "model_top_speed_mph",
        'zeroToSixtytwoMPH': "model_0_to_100_kph",
        'doors': "model_doors",
        'seats': "model_seats",
        'weight_kg': "model_weight_kg",
        'weight_lbs': "model",
        'length_mm': "model_length_mm",
        'length_in': "model_length_in",
        'width_mm': "model_width_mm",
        'width_in': "model_width_in",
        'height_mm': "model_height_mm",
        'height_in': "model_height_in",
        'wheelbase_mm': "model_wheelbase_mm",
        'wheelbase_in': "model_wheelbase_in",
        'fuelEconomyCity_L_100km': "model_lkm_city",
        'fuelEconomyCity_mpg': "model_mpg_city",
        'fuelEconomyHWY_L_100km': "model_lkm_hwy",
        'fuelEconomyHWY_mpg': "model_mpg_hwy",
        'fuelEconomyMixed_L_100km': "model_lkm_mixed",
        'fuelEconomyMixed_mpg': "model_mpg_mixed",
        'fuelCapacity_L': "model_fuel_cap_l",
        'fuelCapacity_g': "model_fuel_cap_g",
        'ModelYear': "model_year",
        'ModelName': "model_name",
        'Make': "make_display"
        }
    # first search for the appropriate year and model to get the model id
    req = requests.get(url1.format(model, year), headers=headers).content
    time.sleep(1)
    ans = req[2:-2]
    ans = json.loads(ans)
    ans = ans["Trims"][0]["model_id"]
    # using the model id: get all data and store it in our dict
    print(url2.format(ans))
    req = requests.get(url2.format(ans), headers=headers).content
    ans = json.loads(req[3:-3])

    for datum in data_car_query:
        datum_alias = data_car_query[datum]
        if datum_alias in ans and str(ans[datum_alias]) != "None":
            data_car_query[datum] = str(ans[datum_alias])
        else:
            data_car_query[datum] = "NULL"
            
    # No kbb data pre 1992
    if year >= 1992:
        # prep our url
        make = data_car_query["Make"]
        base = "https://www.kbb.com/{make}/{model}/{year}/"
        model = model.replace(" ", "-")
        base = base.format(make=make, model=model, year=year)
        # scrape the data
        response = requests.get(base)
        print(response.status_code)
        if response.status_code == 200:
            pageTree = BeautifulSoup(response.content, "html.parser").body
            text = pageTree.find("div", class_="css-1vezu6i-GridContainer").text

            if "We're sorry, our experts haven't reviewed this car yet. Check back with us soon." not in ans:
                text_norm = text.lower()
                pros = text_norm.find("pros")
                cons = text_norm.find("cons")
                whats_new = text_norm.find("what's new")

                price = text_norm.find("price:")
                if price == -1:
                    price = text_norm.find("$")
                end_of_price = text_norm.find("\n", price)

                # slice and dice the text
                pro_text = text[pros:cons].strip()
                cons_text = text[cons:whats_new].strip()
                whats_new_text = text[whats_new:price].strip()
                price_text = text[price:end_of_price].strip()
                price_text = clean_price(price_text)
                review = text[end_of_price:].strip()

                data_kbb["Pros"] = pro_text
                data_kbb["Cons"] = cons_text
                data_kbb["New"] = whats_new_text
                data_kbb["StartingPrice"] = price_text
                data_kbb["Review"] = review

    else:
        print("no kbb data lol")
            
            
            
            
    data_car_query.update(data_kbb)
    return data_car_query

In [7]:
car_spec_kbb = []
exceptions = []
for i, row in craigslist_mmy.iterrows():
    time.sleep(1)
    print((row["alias"], row["Year"]))
    try:
        car_spec_kbb.append(scrape_car_info(row["alias"], row["Year"]))
    except:
        car_spec_kbb.append({})
        exceptions.append((row["alias"], row["Year"]))
results = pd.DataFrame(car_spec_kbb)
results.to_csv("./all_spec_kbb_final.csv", index=False)

('Suburban', 2021)
('Versa', 2021)
('Equinox', 2021)
('Silverado 1500', 2021)
('Sentra', 2021)
('Silverado 3500HD', 2021)
('Corolla', 2021)
('Odyssey', 2021)
('Rogue', 2021)
('XV Crosstrek', 2021)
('Altima', 2021)
('Traverse', 2021)
('Genesis', 2021)
('Silverado 2500HD', 2021)
('Murano', 2021)
('Sierra 2500HD', 2021)


KeyboardInterrupt: 

In [13]:
specs = pd.read_csv("./all_spec_kbb_final.csv")
specs = specs[~specs.OriginCountry.isnull()]
specs = specs.fillna("NULL")

In [10]:
# # craigslist and specs
# final_final = pd.merge(craigslist.copy(), specs.copy(), how="left", left_on=["alias", "Year"], right_on=["ModelName", "ModelYear"] )
# final_final = final_final.drop_duplicates()

In [14]:
craigslist.PostingRegion.unique()

array(["Spokane / Coeur d'Alene", 'Seattle-Tacoma',
       'Kennewick-Pasco-Richland', 'Pullman / Moscow',
       'Skagit Island / San Juan Islands', 'Yakima', 'Bellingham',
       'Wenatchee', 'Moses Lake', 'Olympic Peninsula'], dtype=object)

In [15]:
craigslist.dtypes

CarMake          object
CarModel         object
CarYear           int64
CarPrice          int64
CarCondition     object
MilesOnCar        int64
PostingDate      object
PostingUrl       object
CarImageLinks    object
PostingRegion    object
Model            object
alias            object
Year              int64
dtype: object